In [71]:
import pandas as pd             # data package

import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [72]:
file_path = os.getcwd()

fig_path = file_path +"\\figures"

In [73]:
### STEP1 ###
print("")
print("**********************************************************************************")
print("Downloading and processing BLS file")
print("")

url = "https://data.bls.gov/cew/data/files/2017/csv/2017_annual_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_sf = zf.ZipFile(io.BytesIO(r.content)) 
print('Type of zipfile object:', type(bls_sf))

clist = ['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'disclosure_code', 'annual_avg_estabs',
       'annual_avg_emplvl', 'total_annual_wages','avg_annual_pay']

df = pd.read_csv(bls_sf.open(bls_sf.namelist()[0]), usecols= clist)

# SHOULD PRESPECIFY TYPES TO 

df.head()


**********************************************************************************

Type of zipfile object: <class 'zipfile.ZipFile'>


/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/3036768764.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(bls_sf.open(bls_sf.namelist()[0]), usecols= clist)


,area_fips,own_code,industry_code,agglvl_code,size_code,year,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,avg_annual_pay
0,1000,0,10,50,0,2017,NaN,124881,1936819,89088710816,45997
1,1000,1,10,51,0,2017,NaN,1208,53131,4339038631,81668
2,1000,1,102,52,0,2017,NaN,1208,53131,4339038631,81668
3,1000,1,1021,53,0,2017,NaN,610,11173,716001109,64083
4,1000,1,1022,53,0,2017,NaN,2,12,369309,30354


In [74]:
df.shape
# df.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/countylevel_tariffs_and_exports_initial.csv")

(3561646, 11)

In [75]:
NAICS_county_level = 75 
# This is the code that will select only counties at the 3 digit NAICS level

df_county = df[df.agglvl_code == NAICS_county_level].copy()
df_county.shape

(274608, 11)

In [76]:
df_county = df_county[df_county.own_code == 5]
# Only grab private stuff
df_county.shape
# df_county.head()

(203667, 11)

In [77]:
### Caution ###
df_county['area_fips'] = df['area_fips'].astype(str) # the author forgot this command.
### The author forgot to transform the data type of area_fips and the code in his repository actually do nothing this part. 
### But if this command is run with other commands, there will be no error warning. (I guess that's why he didn't detect this error.)
### So I add this command myself. However, the result is just slightly different.
# df_county.area_fips.dtype
df_county = df_county[(df_county.area_fips.str[0:2] != "72") & (df_county.area_fips.str[0:2] != "78")
              & (df_county.area_fips.str[0:2] != "02") & (df_county.area_fips.str[0:2] != "15")]
# Drop puerto rico, alaska, hawaii...this mayb not be doing what I think it is...as it looks like these guys are there
# Does not matter as analysis is performed withthem, drop them when do the map. 
df_county.head()

,area_fips,own_code,industry_code,agglvl_code,size_code,year,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,avg_annual_pay
3016,1001,5,111,75,0,2017,N,4,0,0,0
3026,1001,5,112,75,0,2017,NaN,3,27,848146,31905
3033,1001,5,113,75,0,2017,NaN,4,36,2901165,80032
3040,1001,5,115,75,0,2017,NaN,9,27,2007731,74132
3052,1001,5,212,75,0,2017,N,4,0,0,0


In [78]:
# This part does not matter at all.
df_county["sup_ind"] = df_county.industry_code.str[1].astype(int)
# sometimes there are super industries floating around we want to drop them.
# not clear if this matters with the conditioning all ready

df_county = df_county[df_county["sup_ind"] > 0]

df_county.area_fips = df_county.area_fips.astype(str)

df_county.shape

(199393, 12)

In [79]:
df_national = df_county.groupby("industry_code").agg({"annual_avg_emplvl": "sum"})
df_national.shape
df_national.head()

,annual_avg_emplvl
industry_code,
111,527459
112,226092
113,45166
114,5005
115,352782


In [80]:
df_national.reset_index(inplace = True)
df_national.rename({"annual_avg_emplvl":"nat_emplvl"}, axis = 1, inplace = True)
df_national.head()

df_national.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step1_pandas.csv") # for checking

In [81]:
df_county.annual_avg_emplvl.sum()

114942685

In [82]:
### STEP2 ###
print("")
print("**********************************************************************************")
print("Downloading and processing Trade Data")
print("")

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

end_use = "hs?get=E_COMMODITY,CTY_CODE,ALL_VAL_MO,CTY_NAME"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2015-01" + "&COMM_LVL=HS6"

url = url + "&CTY_CODE=5700"

r = requests.get(url) 

df_china_trade = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
# Note that the first entry is the labels

df_china_trade.columns = r.json()[0]

df_china_trade.time = pd.to_datetime(df_china_trade.time, format="%Y-%m")
# This is so I can call this correctly...

df_china_trade["china_trade"] = df_china_trade.ALL_VAL_MO.astype(float)

df_china_trade.E_COMMODITY = df_china_trade.E_COMMODITY.astype(str)

df_china_trade.tail(10)


**********************************************************************************



,E_COMMODITY,CTY_CODE,ALL_VAL_MO,CTY_NAME,time,COMM_LVL,CTY_CODE,china_trade
352687,900211,5700,357078,CHINA,2022-10-01,HS6,5700,357078.0
352688,900219,5700,238577,CHINA,2022-10-01,HS6,5700,238577.0
352689,900220,5700,3302121,CHINA,2022-10-01,HS6,5700,3302121.0
352690,900290,5700,1035755,CHINA,2022-10-01,HS6,5700,1035755.0
352691,900311,5700,85241,CHINA,2022-10-01,HS6,5700,85241.0
352692,900319,5700,97029,CHINA,2022-10-01,HS6,5700,97029.0
352693,900390,5700,0,CHINA,2022-10-01,HS6,5700,0.0
352694,900410,5700,94359,CHINA,2022-10-01,HS6,5700,94359.0
352695,900490,5700,215317,CHINA,2022-10-01,HS6,5700,215317.0
352696,900510,5700,25667,CHINA,2022-10-01,HS6,5700,25667.0


In [83]:
df_china_trade.shape

(352697, 8)

In [84]:
df_china_trade.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step2_china.csv")

In [85]:
r

<Response [200]>

In [86]:
end_use = "hs?get=E_COMMODITY,ALL_VAL_MO"

url = "https://api.census.gov/data/timeseries/intltrade/exports/"
url = url + end_use + my_key + "&time==from+2015-01" + "&COMM_LVL=HS6"

r = requests.get(url) 

df_all_trade = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
# Note that the first entry is the labels

df_all_trade.columns = r.json()[0]

df_all_trade.time = pd.to_datetime(df_all_trade.time, format="%Y-%m")
# This is so I can call this correctly...

df_all_trade["total_trade"] = df_all_trade.ALL_VAL_MO.astype(float)

df_all_trade.E_COMMODITY = df_all_trade.E_COMMODITY.astype(str)

df_all_trade.head(10)

,E_COMMODITY,ALL_VAL_MO,time,COMM_LVL,total_trade
0,600544,4546,2015-01-01,HS6,4546.0
1,600542,53239,2015-01-01,HS6,53239.0
2,600541,8175,2015-01-01,HS6,8175.0
3,600534,497405,2015-01-01,HS6,497405.0
4,600533,1993074,2015-01-01,HS6,1993074.0
5,600532,10793848,2015-01-01,HS6,10793848.0
6,600531,1621489,2015-01-01,HS6,1621489.0
7,600523,33574,2015-01-01,HS6,33574.0
8,600522,1305273,2015-01-01,HS6,1305273.0
9,600521,146116,2015-01-01,HS6,146116.0


In [87]:
df_all_trade.shape

(492684, 5)

In [88]:
df_all_trade.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step2_world.csv")

In [89]:
dftrade = df_all_trade.merge(df_china_trade[["E_COMMODITY", "time","china_trade"]], left_on = ["E_COMMODITY", "time"], 
                             right_on = ["E_COMMODITY", "time"], how = "left")

dftrade.set_index("time", inplace = True)

dftrade.drop(["ALL_VAL_MO"], axis = 1, inplace = True)

In [90]:
dftrade.head()

,E_COMMODITY,COMM_LVL,total_trade,china_trade
time,,,,
2015-01-01,600544,HS6,4546.0,NaN
2015-01-01,600542,HS6,53239.0,NaN
2015-01-01,600541,HS6,8175.0,NaN
2015-01-01,600534,HS6,497405.0,NaN
2015-01-01,600533,HS6,1993074.0,NaN


In [91]:
dftrade.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step2_pandas.csv") # for checking

In [92]:
### STEP 3 ###
dftrade_17 = dftrade.loc["2017"].groupby("E_COMMODITY").agg({"china_trade":"sum"})
dftrade_17.head(10)

,china_trade
E_COMMODITY,
010121,29098.0
010129,27975.0
010130,8340.0
010190,0.0
010221,80280.0
010229,2682.0
010231,3806.0
010239,5000.0
010290,0.0


In [93]:
#url = "https://www.census.gov/foreign-trade/reference/codes/concordance/expconcord17.xls"

#df_concordance = pd.read_excel(url, dtype = {"commodity": str, "naics": str})

#df_concordance["hs8"] = df_concordance.commodity.str[0:8]
# truncate down to get the hs8

#df_concordance["hs6"] = df_concordance.commodity.str[0:6]
# truncate down to get the hs6

#df_concordance["naics3"] = df_concordance["naics"].str[0:3]

#dict_concordance = dict(zip(df_concordance.hs6,df_concordance.naics)) 

# This creates a dictionaty from which we can map the hs6 to the naics codes

# Full disclosure. There is an issue with the creation of the dictionary as a unique 
# mapping from hs6 to naics. The notebook ``alt_hs_naics_mapping.ipynb'' provides a complete discussion.
# Ultimatly, this does not matter for the results (relative to the alternative below)

# Below is a fix/alternative approach to creating the mapping from hs6 to naics. In the
# cases where there are multiple naics codes for each hs6 code, it assigns the naics code that is
# associated with the most trade. 

# For future reference check this out: https://www.bea.gov/industry/zip/NDN0317.zip

file_path = os.getcwd()

alt_concordance = pq.read_table(file_path + "\\data\\alt_concordance.parquet").to_pandas()

alt_concordance.head()

dict_concordance = dict(zip(alt_concordance.hs6,alt_concordance.naics))

In [94]:
dftrade_17["hs6"] = dftrade_17.index

dftrade_17["naics"] = dftrade_17["hs6"].map(dict_concordance)

dftrade_17["naics4"] = dftrade_17["naics"].str[0:4]

dftrade_17["naics3"] = dftrade_17["naics"].str[0:3]

dftrade_17.rename({"china_trade":"2017_china_trade"}, axis = 1, inplace = True)

In [95]:
dftrade_17.head()

,2017_china_trade,hs6,naics,naics4,naics3
E_COMMODITY,,,,,
010121,29098.0,010121,112920,1129,112
010129,27975.0,010129,112920,1129,112
010130,8340.0,010130,112920,1129,112
010190,0.0,010190,112920,1129,112
010221,80280.0,010221,11211X,1121,112


In [96]:
dftrade_17_naics3 = dftrade_17.groupby("naics3").agg({"2017_china_trade": "sum"})
dftrade_17_naics3.head()

,2017_china_trade
naics3,
111,1.578548e+10
112,1.066820e+08
113,1.316022e+09
114,1.322142e+09
211,6.730487e+09


In [97]:
df_national = df_national.merge(dftrade_17_naics3["2017_china_trade"],
                                left_on = "industry_code", right_index = True, how = "left")

In [98]:
df_national

,industry_code,nat_emplvl,2017_china_trade
0,111,527459,1.578548e+10
1,112,226092,1.066820e+08
2,113,45166,1.316022e+09
3,114,5005,1.322142e+09
4,115,352782,NaN
5,211,103960,6.730487e+09
6,212,88420,1.745069e+09
7,213,268573,NaN
8,221,414125,NaN
9,236,1480653,NaN


In [99]:
# create the "weights" by 
df_national["2017_china_trade"].replace(np.nan, 0, inplace = True)

df_national["trd_wts"] = (df_national["2017_china_trade"]/df_national["2017_china_trade"].sum())

In [100]:
df_national.trd_wts.sum()

1.0

In [101]:
df_national

,industry_code,nat_emplvl,2017_china_trade,trd_wts
0,111,527459,1.578548e+10,0.128477
1,112,226092,1.066820e+08,0.000868
2,113,45166,1.316022e+09,0.010711
3,114,5005,1.322142e+09,0.010761
4,115,352782,0.000000e+00,0.000000
5,211,103960,6.730487e+09,0.054779
6,212,88420,1.745069e+09,0.014203
7,213,268573,0.000000e+00,0.000000
8,221,414125,0.000000e+00,0.000000
9,236,1480653,0.000000e+00,0.000000


In [102]:
df_national.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step3_pandas.csv") # for checking

In [103]:
### STEP 4 ### 
# While the author put this cell in the bottom of STEP 3, I think this belongs to the preparation of STEP 4.

dftrade["hs6"] = dftrade.E_COMMODITY

dftrade["naics"] = dftrade["hs6"].map(dict_concordance)

dftrade["naics4"] = dftrade["naics"].str[0:4]

dftrade["naics3"] = dftrade["naics"].str[0:3]

In [104]:
dftrade

,E_COMMODITY,COMM_LVL,total_trade,china_trade,hs6,naics,naics4,naics3
time,,,,,,,,
2015-01-01,600544,HS6,4546.0,NaN,600544,313240,3132,313
2015-01-01,600542,HS6,53239.0,NaN,600542,313240,3132,313
2015-01-01,600541,HS6,8175.0,NaN,600541,313240,3132,313
2015-01-01,600534,HS6,497405.0,NaN,600534,NaN,NaN,NaN
2015-01-01,600533,HS6,1993074.0,NaN,600533,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-10-01,200880,HS6,602495.0,NaN,200880,311421,3114,311
2022-10-01,200891,HS6,4597.0,0.0,200891,311421,3114,311
2022-10-01,200893,HS6,24266808.0,5074422.0,200893,311421,3114,311


In [105]:
# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_1.dta")

In [106]:
#tariffs = pd.read_csv("tariff_list_naics.csv", dtype = {"HS-8 code": str,"HS6": str,"naics": str,"naics4": str})

# bring in the tariff data

tariffs = pd.read_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/updated_tariff_data/new_tariff_list_max.csv", dtype = {"hs6": str})
# This is the tariff dataset created by updated_tariff_data.ipynb (note the max tariff means taking the largest value
# when going from Chinese hs10 to hs6. This does not matter, if anything gives more conservative resutls)

tariffs.time_of_tariff = pd.to_datetime(tariffs.time_of_tariff, format="%Y-%m")
# make sure the time is there.

tariffs.set_index("time_of_tariff",inplace = True)
tariffs

,hs6,tariff
time_of_tariff,,
2018-01-01,010121,0.0
2018-04-02,010121,0.0
2018-05-01,010121,0.0
2018-07-01,010121,0.0
2018-07-06,010121,0.0
...,...,...
2018-08-23,970600,0.0
2018-09-24,970600,10.0
2018-11-01,970600,10.0


In [107]:
# create dictionaries to do the following " you tell me HS, I tell you tariff" by time.

initial_tariff = dict(zip(tariffs.loc["2018-01-01"].hs6,tariffs.loc["2018-01-01"].tariff)) 
# These are the initial mfn tariffs. We will apply these from 2017 up untill the changes...

tariff_dict_232 = dict(zip(tariffs.loc["2018-04-02"].hs6,tariffs.loc["2018-04-02"].tariff)) 
# These are the section 232 tariffs, response to US steel and aluminum. 

tariff_dict_r1 = dict(zip(tariffs.loc["2018-07-06"].hs6,tariffs.loc["2018-07-06"].tariff)) 
# tariff_dict_r1 = dict(zip(tariffs.loc["2018-07-06"].HS6,tariffs.loc["2018-07-06"].tariff)) 
# This will create a mapping from HS6 to tariff, you tell me HS, I tell you tariff

tariff_dict_r2 = dict(zip(tariffs.loc[dt.datetime(2018,8,23)].hs6,tariffs.loc[dt.datetime(2018,8,23)].tariff)) 

#tariff_dict_r2 = dict(zip(tariffs.loc[dt.datetime(2018,8,23)].HS6,tariffs.loc[dt.datetime(2018,8,23)].tariff)) 
# This will create a mapping from HS6 to tariff, you tell me HS, I tell you tariff, round 2

tariff_dict_r3 = dict(zip(tariffs.loc[dt.datetime(2018,9,24)].hs6,tariffs.loc[dt.datetime(2018,9,24)].tariff))
#tariff_dict_r3 = dict(zip(tariffs.loc[dt.datetime(2018,9,24)].HS6,tariffs.loc[dt.datetime(2018,9,24)].tariff)) 
# This will create a mapping from HS6 to tariff, you tell me HS, I tell you tariff, round 3

tariff_dict_mfn = dict(zip(tariffs.loc["2018-11-01"].hs6,tariffs.loc["2018-11-01"].tariff))
# This reflects mfn adjustments that China made later in the year.

tariff_dict_mfn_2019 = dict(zip(tariffs.loc["2019-01-02"].hs6,tariffs.loc["2019-01-02"].tariff))
# This reflects mfn adjustments and auto adjustment China made at the start of 2019

In [108]:
dftrade["tariff"] = 0

# Then use the map function which will fill in the tariff in the correct places..
# Note the key issue was that the dictionaries were overwritting stuff, hence
# the update...

In [109]:
# Start with the MFN...
dftrade.loc["2015-01":,"tariff"] = dftrade.loc["2015-01":,"hs6"].map(initial_tariff)
# print("done")

# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_2.dta")

In [110]:
# Now update given the 232 response
initial_tariff.update(tariff_dict_232)

dftrade.loc["2018-04":,"tariff"] = dftrade.loc["2018-04":,"hs6"].map(initial_tariff)
# print("done")

# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_3.dta")

In [111]:
# This is the big Phase 1 of the war
initial_tariff.update(tariff_dict_r1)

dftrade.loc["2018-07":,"tariff"] = dftrade.loc["2018-07":,"hs6"].map(initial_tariff)
#print("done")

# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_4.dta")

In [112]:
# Here is phase 2
initial_tariff.update(tariff_dict_r2)

dftrade.loc["2018-09":,"tariff"] = dftrade.loc["2018-09":,"hs6"].map(initial_tariff)
#print("done")

# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_5.dta")

In [113]:
# Here is phase 3
initial_tariff.update(tariff_dict_r3)

dftrade.loc["2018-10":,"tariff"] = dftrade.loc["2018-10":,"hs6"].map(initial_tariff)

#print("done")

# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_6.dta")

In [114]:
# China then adjusts the mfn
initial_tariff.update(tariff_dict_mfn)

dftrade.loc["2018-11":,"tariff"] = dftrade.loc["2018-11":,"hs6"].map(initial_tariff)

#print("done")
# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_7.dta")

In [115]:
# An update on the mfn's
initial_tariff.update(tariff_dict_mfn_2019)

dftrade.loc["2019-01":,"tariff"] = dftrade.loc["2019-01":,"hs6"].map(initial_tariff)

# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_8.dta")

In [116]:
dftrade.shape

(492684, 9)

In [117]:
dftrade["tariff"] = dftrade["tariff"].replace(np.nan,0)

dftrade[dftrade["tariff"] == 25].head()

dftrade["2018-08"].head(25)

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1580050427.py:5: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  dftrade["2018-08"].head(25)


,E_COMMODITY,COMM_LVL,total_trade,china_trade,hs6,naics,naics4,naics3,tariff
time,,,,,,,,,
2018-08-01,630492,HS6,677456.0,0.0,630492,314120,3141,314,6.0
2018-08-01,630491,HS6,759183.0,0.0,630491,314120,3141,314,6.0
2018-08-01,630493,HS6,993665.0,0.0,630493,314120,3141,314,6.0
2018-08-01,630499,HS6,607956.0,NaN,630499,314120,3141,314,6.0
2018-08-01,630510,HS6,128501.0,0.0,630510,314910,3149,314,4.0
2018-08-01,630520,HS6,142000.0,NaN,630520,314910,3149,314,6.0
2018-08-01,630532,HS6,2934809.0,22689.0,630532,314910,3149,314,6.0
2018-08-01,630533,HS6,1667727.0,49962.0,630533,314910,3149,314,6.0
2018-08-01,630539,HS6,4727700.0,92093.0,630539,314910,3149,314,6.0


In [118]:
dftrade = dftrade.merge(dftrade_17["2017_china_trade"], how = "inner", left_on = "E_COMMODITY", right_index = True)

In [119]:
dftrade.head()

,E_COMMODITY,COMM_LVL,total_trade,china_trade,hs6,naics,naics4,naics3,tariff,2017_china_trade
time,,,,,,,,,,
2015-01-01,600544,HS6,4546.0,NaN,600544,313240,3132,313,10.0,0.0
2015-02-01,600544,HS6,3221.0,NaN,600544,313240,3132,313,10.0,0.0
2015-03-01,600544,HS6,0.0,NaN,600544,313240,3132,313,10.0,0.0
2015-04-01,600544,HS6,0.0,NaN,600544,313240,3132,313,10.0,0.0
2015-05-01,600544,HS6,0.0,NaN,600544,313240,3132,313,10.0,0.0


In [120]:
# dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/test_4_9.dta")

In [121]:
def trd_weighted_avg(df):
    # A function to create the trade weighted average of the tariff rates
    # by round...
    
    trd_w_avg = df["tariff"].multiply(df["2017_china_trade"],axis = 0).sum() 
    # here wuse the 2017 annual values to weight it
    
    trd_w_avg = trd_w_avg / df["2017_china_trade"].sum()
    
    
    foo = {"tariff_trd_w_avg": [trd_w_avg ], 
           "total_trade": df["total_trade"].sum(),
          "china_trade" : df["china_trade"].sum()}
    
    return pd.DataFrame(foo)

In [122]:
grp = dftrade.groupby(["time","naics3"])

exp_trf_bynaics = grp.apply(trd_weighted_avg)

exp_trf_bynaics = exp_trf_bynaics.droplevel(2)

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1189988801.py:8: RuntimeWarning: invalid value encountered in double_scalars
  trd_w_avg = trd_w_avg / df["2017_china_trade"].sum()


In [123]:
exp_trf_bynaics

tariff_trd_w_avg   total_trade   china_trade
time       naics3                                              
2015-01-01 111             7.731281  6.628535e+09  2.260464e+09
           112            11.923828  1.415040e+08  3.176711e+06
           113             0.173604  1.617550e+08  5.432390e+07
           114             8.184240  1.774655e+08  3.002359e+07
           211             0.647145  1.728807e+09  2.445570e+07
...                             ...           ...           ...
2022-10-01 339            15.972921  8.506537e+09  4.960927e+08
           910            30.479322  2.248486e+09  1.648746e+08
           930            15.003084  5.893777e+08  4.739298e+07
           980                  NaN  8.066345e+06  0.000000e+00
           990             0.155066  5.766914e+09  2.103216e+08

[2914 rows x 3 columns]

In [124]:
exp_trf_bynaics.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step4_pandas.csv")

In [125]:
# These two rows make no difference
exp_trf_bynaics["time"] = exp_trf_bynaics.index.get_level_values(0)
exp_trf_bynaics = exp_trf_bynaics.droplevel(level=0)

In [167]:
print("")
print("**********************************************************************************")
print("Constructing County-Level Tariffs and Exports")
print("")

grp = df_county.groupby("area_fips") # This creates groups at the county level.

grp.get_group("1001").head() # Let's just look at one of the groups...


**********************************************************************************
Constructing County-Level Tariffs and Exports



,area_fips,own_code,industry_code,agglvl_code,size_code,year,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,avg_annual_pay,sup_ind
3016,1001,5,111,75,0,2017,N,4,0,0,0,1
3026,1001,5,112,75,0,2017,NaN,3,27,848146,31905,1
3033,1001,5,113,75,0,2017,NaN,4,36,2901165,80032,1
3040,1001,5,115,75,0,2017,NaN,9,27,2007731,74132,1
3052,1001,5,212,75,0,2017,N,4,0,0,0,1


In [168]:
def create_trade_weights(df):
    # Takes in the county groupings and will return, for each county, a time series of export
    # exposure, tariffs, and other statistics. 

    new_df = df.merge(df_national[["nat_emplvl",
                                   "industry_code", "trd_wts"]],
                                  how = "outer", left_on = "industry_code", right_on = "industry_code")
    # Merge the nation with the county, why, we want to make sure all the naics codes are lined up properly
        
    new_df["emp_wts"] = (new_df.annual_avg_emplvl/new_df.nat_emplvl)
     
    # create the weights...
        
    foo_df = exp_trf_bynaics.merge(new_df[["emp_wts","trd_wts",
                                           "industry_code",
                                          "annual_avg_emplvl"]], left_index = True, right_on = "industry_code")  
    
    # Now each weight is for a NAICS code, we will merge it with the export trade data set, so for all naics, all time...
    # This is a big df whith all trade data and then the county's weights for each naics code
    
    foo_grp = foo_df.groupby("time")
    
    # group by time. 
    
    foo = foo_grp.apply(trade_by_naics)
    
    # Then for each time gropuing, we aggregate across the naics codes according to the weights above.
    
    foo = foo.droplevel(1)
    
    foo["fips"] = df["area_fips"].astype(str).iloc[0]
    
    # some cleaning of the df
    
    foo["total_employment"] = new_df.annual_avg_emplvl.sum()
    
    # get total employment.
    
    return pd.DataFrame(foo)

In [169]:
def trade_by_naics(df):
    # Simple function just to test about aggregation 

    china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
    
    total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
    # the first term multiplies trade by the county's share of national level employment
    # then the outside term divides by number of workers in a county. 
    
    #tariff_nwt_pc = (1/df["annual_avg_emplvl"].sum())*(df["tariff_trd_w_avg"]*df["emp_wts"]).sum()
    # This is the measure that makes most sense, need to justify it...
    tariff =  ((df["annual_avg_emplvl"]*df["tariff_trd_w_avg"])/df["annual_avg_emplvl"].sum()).sum()
    # local employment share weighted tariff. So if all guys are in area are working in soy,
    # then they are facing the soybean tariff....
    
    foo = {"total_exp_pc": [total_exp_pc],
          "china_exp_pc": [china_exp_pc],
           "tariff": [tariff],
          "emplvl_2017": df["annual_avg_emplvl"].sum()}

    return pd.DataFrame(foo)

In [170]:
trade_county = grp.apply(create_trade_weights)

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

...

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1

/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:4: RuntimeWarning: invalid value encountered in double_scalars
  china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()
/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_99332/1230872137.py:6: RuntimeWarning: invalid value encountered in double_scalars
  total_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["total_trade"]*df["emp_wts"]).sum()


In [171]:
trade_county.sort_values(by = ["tariff","emplvl_2017"], ascending = False).head(25)

,,total_exp_pc,china_exp_pc,tariff,emplvl_2017,fips,total_employment
area_fips,time,,,,,,
29133,2021-01-01,16965.243217,6430.407891,32.843891,373.0,29133,1890.0
28143,2021-01-01,16965.243217,6430.407891,32.843891,311.0,28143,5813.0
28011,2021-01-01,16965.243217,6430.407891,32.843891,286.0,28011,5285.0
28027,2021-01-01,16965.243217,6430.407891,32.843891,275.0,28027,4471.0
4012,2021-01-01,16965.243217,6430.407891,32.843891,258.0,4012,2180.0
29143,2021-01-01,16965.243217,6430.407891,32.843891,227.0,29143,3487.0
53043,2021-01-01,16965.243217,6430.407891,32.843891,227.0,53043,1132.0
13101,2021-01-01,16965.243217,6430.407891,32.843891,216.0,13101,411.0
8109,2021-01-01,16965.243217,6430.407891,32.843891,216.0,8109,699.0


In [172]:
# One more adjustment
trade_county["tariff"] = (trade_county["emplvl_2017"]/
                              trade_county["total_employment"])*trade_county["tariff"]

trade_county["china_exp_pc"] = (trade_county["emplvl_2017"]/
                                    trade_county["total_employment"])*trade_county["china_exp_pc"]

trade_county["total_exp_pc"] = (trade_county["emplvl_2017"]/
                                    trade_county["total_employment"])*trade_county["total_exp_pc"]

In [173]:
os.getcwd()

'/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports'

In [174]:
os.getcwd()

file_path = os.getcwd() + "\\data"+ "\\total_trade_data_2015.parquet"

pq.write_table(pa.Table.from_pandas(trade_county.reset_index()), file_path)

In [175]:
trade_county.sort_values(by = ["tariff","emplvl_2017"], ascending = False).head(50)

total_exp_pc  china_exp_pc     tariff  emplvl_2017  \
area_fips time                                                             
53023     2021-01-01  16965.243217   6430.407891  32.843891         37.0   
28055     2021-01-01  16965.243217   6430.407891  32.843891         21.0   
53023     2019-03-01  11109.613272   1311.000902  32.843845         37.0   
          2019-04-01   9905.368999    746.478923  32.843845         37.0   
          2019-05-01   9732.673897   1233.610952  32.843845         37.0   
28055     2019-03-01  11109.613272   1311.000902  32.843845         21.0   
          2019-04-01   9905.368999    746.478923  32.843845         21.0   
          2019-05-01   9732.673897   1233.610952  32.843845         21.0   
53023     2019-02-01  10570.660192   1598.169416  32.843843         37.0   
          2020-01-01  10423.150262   1695.383927  32.843843         37.0   
28055     2019-02-01  10570.660192   1598.169416  32.843843         21.0   
          2020-01-01  10423.150262   1695.383927  32.843843         21.0   
53023     2018-10-01  11231.049517    346.138693  32.843691         37.0   
          2018-11-01  11249.504119    248.883420  32.843691         37.0   
          2018-12-01  10476.631200    329.614804  32.843691         37.0   
          2019-06-01   9301.735854   1616.037222  32.843691         37.0   
          2019-07-01   9216.108647   1572.019873  32.843691         37.0   
          2019-08-01   9483.574685   2125.852800  32.843691         37.0   
          2019-09-01   8124.844553    914.928690  32.843691         37.0   
          2019-10-01  10824.622429   1542.532286  32.843691         37.0   
          2019-11-01  12016.009318   3536.937616  32.843691         37.0   
          2019-12-01  11117.626155   2235.790659  32.843691         37.0   
          2020-03-01  10110.653323    622.481091  32.843691         37.0   
          2020-04-01   8909.063766    963.923101  32.843691         37.0   
          2020-05-01   8880.967084    993.981765  32.843691         37.0   
          2020-06-01   8385.816968   1092.490267  32.843691         37.0   
          2020-07-01   8523.712038   1240.559626  32.843691         37.0   
          2020-08-01   9764.438870   2840.383035  32.843691         37.0   
          2020-09-01  11581.549538   4609.846498  32.843691         37.0   
          2020-10-01  16581.578691   8172.253095  32.843691         37.0   
          2020-11-01  17202.469589   8234.370780  32.843691         37.0   
          2020-12-01  17716.690054   7707.722577  32.843691         37.0   
          2021-02-01  13826.899882   2689.090453  32.843691         37.0   
          2021-03-01  14200.819611   2354.690012  32.843691         37.0   
          2021-04-01  12857.520486   2729.342902  32.843691         37.0   
          2021-05-01  13098.204987   2800.014892  32.843691         37.0   
          2021-06-01  10618.492361   2116.162466  32.843691         37.0   
          2021-07-01  10258.662529   2380.358045  32.843691         37.0   
          2021-08-01   8948.958715   1694.678633  32.843691         37.0   
          2021-09-01   9031.069761   1820.199140  32.843691         37.0   
          2021-10-01  18815.454722   8749.865241  32.843691         37.0   
          2021-11-01  19341.639257   8442.318895  32.843691         37.0   
          2021-12-01  16507.400257   5540.395576  32.843691         37.0   
28055     2018-10-01  11231.049517    346.138693  32.843691         21.0   
          2018-11-01  11249.504119    248.883420  32.843691         21.0   
          2018-12-01  10476.631200    329.614804  32.843691         21.0   
          2019-06-01   9301.735854   1616.037222  32.843691         21.0   
          2019-07-01   9216.108647   1572.019873  32.843691         21.0   
          2019-08-01   9483.574685   2125.852800  32.843691         21.0   
          2019-09-01   8124.844553    914.928690  32.843691         21.0   

                       fips  total_employment  
area_fips time       

In [176]:
trade_county.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/trade_county_pandas.csv")

In [163]:
exposure = pd.qcut(trade_county.xs('2015-1-1', level=1).tariff, 4 ,labels = False)

In [166]:
exposure

area_fips
10001    2.0
10003    0.0
10005    3.0
1001     2.0
1003     1.0
        ... 
9009     2.0
9011     1.0
9013     2.0
9015     3.0
9999     0.0
Name: tariff, Length: 3185, dtype: float64

In [164]:
most_exposed = exposure[exposure == 3].index.tolist()

trade_county.loc[most_exposed].xs('2018-12-1', level=1).tariff.mean()

6.25370998592832

In [165]:
exposure.head()

area_fips
10001    2.0
10003    0.0
10005    3.0
1001     2.0
1003     1.0
Name: tariff, dtype: float64